In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/ratings_small.csv


In [75]:
meta = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
meta = meta[ ['id', 'original_title', 'original_language', 'genres'] ]
meta = meta.rename(columns={'id':'movieId', 'original_title': 'title', 'original_language': 'language'})
meta = meta.loc[meta['language'] == 'en',:]
meta.movieId = pd.to_numeric(meta.movieId)

meta

,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45459,222848,Caged Heat 3000,en,"[{'id': 878, 'name': 'Science Fiction'}]"
45460,30840,Robin Hood,en,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satana likuyushchiy,en,[]


In [76]:
def str_to_set(x):
    genre_set = set()
    for item in eval(x):
        genre_set.add(item['name'])
    return genre_set

meta.genres = meta.genres.apply(str_to_set)

meta

,movieId,title,language,genres
0,862,Toy Story,en,"{Comedy, Animation, Family}"
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}"
2,15602,Grumpier Old Men,en,"{Comedy, Romance}"
3,31357,Waiting to Exhale,en,"{Comedy, Romance, Drama}"
4,11862,Father of the Bride Part II,en,{Comedy}
...,...,...,...,...
45459,222848,Caged Heat 3000,en,{Science Fiction}
45460,30840,Robin Hood,en,"{Action, Drama, Romance}"
45463,67758,Betrayal,en,"{Action, Drama, Thriller}"
45464,227506,Satana likuyushchiy,en,{}


In [77]:
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
keywords.keywords = keywords.keywords.apply(str_to_set)
keywords = keywords.rename(columns={'id': 'movieId'})
keywords.movieId = pd.to_numeric(keywords.movieId)
keywords

,movieId,keywords
0,862,"{new toy, toy comes to life, boy, rivalry, toy..."
1,8844,"{new home, recluse, disappearance, giant insec..."
2,15602,"{best friend, fishing, duringcreditsstinger, o..."
3,31357,"{based on novel, chick flick, interracial rela..."
4,11862,"{pregnancy, aging, midlife crisis, contracepti..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{artist, pinoy, play}"
46416,67758,{}
46417,227506,{}


In [78]:
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Comedy, Animation, Family}","{new toy, toy comes to life, boy, rivalry, toy..."
1,8844,Jumanji,en,"{Adventure, Family, Fantasy}","{new home, recluse, disappearance, giant insec..."
2,15602,Grumpier Old Men,en,"{Comedy, Romance}","{best friend, fishing, duringcreditsstinger, o..."
3,31357,Waiting to Exhale,en,"{Comedy, Romance, Drama}","{based on novel, chick flick, interracial rela..."
4,11862,Father of the Bride Part II,en,{Comedy},"{pregnancy, aging, midlife crisis, contracepti..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Action, Drama, Romance}",{}
32849,67758,Betrayal,en,"{Action, Drama, Thriller}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [79]:
def jaccard_similarity(s1, s2):
    if len(s1|s2) == 0:
        return 0
    return len(s1&s2)/len(s1|s2)

dk = meta.loc[meta.title == 'The Dark Knight'].iloc[0]
dkr = meta.loc[meta.title == 'The Dark Knight Rises'].iloc[0]
jaccard_similarity(dk.genres|dk.keywords, dkr.genres|dkr.keywords)

0.37142857142857144

In [80]:
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings.movieId = pd.to_numeric(ratings.movieId)
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')
ratings

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
33295,652,127728,5.0,1439586990,8:46
33296,652,129009,4.0,1442690827,Love Is a Ball
33297,653,2103,3.0,948161066,Solaris
33298,659,167,4.0,836137550,K-PAX


In [81]:
matrix = ratings.pivot_table(index='userId', columns='title', values='rating')
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
def pearson_similarity(u1, u2):
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    if np.sum(u1_c ** 2) * np.sum(u2_c ** 2) != 0:
        return np.sum(u1_c * u2_c)/np.sqrt( np.sum(u1_c ** 2) * np.sum(u2_c ** 2) )
    else:
        return 0

In [83]:
# ratings.loc[ratings.title.isin(['Prom Night', 'The Dark Knight'])]
dk_rating = matrix['The Dark Knight']
pk_rating = matrix['Prom Night']
pearson_similarity(dk_rating, pk_rating)


0.773565934694095

In [84]:
def find_similar_movies(input_title, matrix, n, alpha):
    input_meta = meta.loc[ meta['title'] == input_title].iloc[0]
    input_set = input_meta.genres | input_meta.keywords
    
    result = []
    
    for this_title in matrix.columns:
        if this_title == input_title:
            continue

        this_meta = meta.loc[ meta['title'] == this_title].iloc[0]
        this_set = this_meta.genres | this_meta.keywords
        
        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        jaccard = jaccard_similarity(this_set, input_set)
        
        score = alpha * pearson + (1-alpha) * jaccard
        result.append( (this_title, pearson, jaccard, score) )
        
    result.sort(key=lambda r: r[3], reverse=True)
    
    return result[:n]

In [87]:
result = find_similar_movies('The Dark Knight', matrix, 10, 0.3)

pd.DataFrame(result, columns=['title', 'pearson', 'jaccard', 'score'])

,title,pearson,jaccard,score
0,Wild Wild West,0.773566,0.032258,0.254650
1,Prom Night,0.773566,0.022222,0.247625
2,Batman Begins,0.005015,0.292683,0.206383
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.200643
4,Blue Thunder,0.326617,0.133333,0.191318
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.189930
6,Topaz,0.377145,0.103448,0.185557
7,Big Bad Mama,0.344649,0.107143,0.178395
8,Sneakers,0.415830,0.068966,0.173025
9,The Enforcer,0.326617,0.103448,0.170399
